##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Analisador de modelos do TensorFlow Lite

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/lite/guide/model_analyzer"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/lite/guide/model_analyzer.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

A API TensorFlow Lite Model Analyzer (analisador de modelos do TF Lite) ajuda a analisar modelos no formato do TensorFlow Lite por meio da listagem da estrutura do modelo.


## API Model Analyzer

A seguinte API está disponível para o TensorFlow Lite Model Analyzer.

```
tf.lite.experimental.Analyzer.analyze(model_path=None,
                                      model_content=None,
                                      gpu_compatibility=False)
```

Confira os detalhes da API em https://www.tensorflow.org/api_docs/python/tf/lite/experimental/Analyzer ou executando `help(tf.lite.experimental.Analyzer.analyze)` em um terminal Python.


## Uso básico com um modelo simples do Keras

O código abaixo mostra o uso básico do Model Analyzer, indicando o conteúdo do modelo do Keras convertido para o conteúdo de modelo do TF Lite, formatado como um objeto flatbuffer.

In [ ]:
import tensorflow as tf

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)


## Uso básico com o modelo MobileNetV3Large do Keras

Essa API funciona com modelos grandes, como o MobileNetV3Large. Como a saída é grande, talvez seja melhor explorá-la com seu editor de texto favorito.

In [ ]:
model = tf.keras.applications.MobileNetV3Large()
fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model)

## Verifique a compatibilidade com delegado de GPU

A API ModelAnalyzer conta com uma maneira de verificar a compatibilidade com [delegado de GPU](https://www.tensorflow.org/lite/performance/gpu) do modelo fornecido por meio da opção `gpu_compatibility=True`.


### Caso 1 – Quando o modelo é incompatível

O código abaixo mostra uma maneira de usar a opção `gpu_compatibility=True` para uma tf.function simples que usa `tf.slice` com um tensor bidimensional e `tf.cosh`, que não são compatíveis com delegado de GPU.

Será exibido o aviso `GPU COMPATIBILITY WARNING` para cada nó com problemas de compatibilidade.

In [ ]:
import tensorflow as tf

@tf.function(input_signature=[
    tf.TensorSpec(shape=[4, 4], dtype=tf.float32)
])
def func(x):
  return tf.cosh(x) + tf.slice(x, [1, 1], [1, 1])

converter = tf.lite.TFLiteConverter.from_concrete_functions(
    [func.get_concrete_function()], func)
converter.target_spec.supported_ops = [
    tf.lite.OpsSet.TFLITE_BUILTINS,
    tf.lite.OpsSet.SELECT_TF_OPS,
]
fb_model = converter.convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)

### Caso 2 – Quando o modelo é compatível

Neste exemplo, o model fornecido é compatível com delegado de GPU.

**Observação:** embora a ferramenta não encontre nenhum problema de compatibilidade, ela não garante que seu modelo funcione bem com delegado de GPU em todos os dispositivos. Pode haver incompatibilidade em tempo de execução, como ausência do recurso `CL_DEVICE_IMAGE_SUPPORT` no back-end OpenGL alvo.


In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(128, 128)),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

fb_model = tf.lite.TFLiteConverter.from_keras_model(model).convert()

tf.lite.experimental.Analyzer.analyze(model_content=fb_model, gpu_compatibility=True)